In [3]:
# Importing Packages and Data
import pandas as pd
import numpy as np
import biogeme as biogeme
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme import models
import biogeme.messaging as msgnotes
from biogeme.expressions import Beta
from biogeme.expressions import (
    Beta,
    DefineVariable,
    bioDraws,
    PanelLikelihoodTrajectory,
    MonteCarlo,
    log,
    Derive,
    bioNormalCdf,
    Elem
)
# from biogeme.results import calculate_correlation
import math
from datetime import datetime
import matplotlib.pyplot as plt

In [4]:
# Import micromobility data
micro_pool_socio = pd.read_csv('Data/micro_pool_socio_bio2up.csv')

In [5]:
# make new column: PRCP_YES [PRCP > 1]
# make new column: BKLN_YES [BKLN > 1]

micro_pool_socio['PRCP_YES'] = np.where(micro_pool_socio['PRCP'] > 1, 1, 0)
micro_pool_socio['BKLN_YES'] = np.where(micro_pool_socio['BKLN'] > 1, 1, 0)

database = db.Database('micro_pool', micro_pool_socio)
database.panel("who")
globals().update(database.variables)


In [6]:
ASC_CAR = Beta('ASC_CAR', 0, None, None, 1)
ASC_CAR_S = Beta('ASC_CAR_S', 1, None, None, 0)
ASC_CAR_RND = ASC_CAR + ASC_CAR_S * bioDraws('ASC_CAR_RND', 'NORMAL_HALTON3')

ASC_TRANSIT = Beta('ASC_TRANSIT', 0, None, None, 0)
ASC_TRANSIT_S = Beta('ASC_TRANSIT_S', 1, None, None, 0)
ASC_TRANSIT_RND = ASC_TRANSIT + ASC_TRANSIT_S * bioDraws('ASC_TRANSIT_RND', 'NORMAL_HALTON3')

ASC_RH = Beta('ASC_RH', 0, None, None, 0)
ASC_RH_S = Beta('ASC_RH_S', 1, None, None, 0)
ASC_RH_RND = ASC_RH + ASC_RH_S * bioDraws('ASC_RH_RND', 'NORMAL_HALTON3')

# ASC_SCOOTER = Beta('ASC_SCOOTER', 0, None, None, 0)
# ASC_SCOOTER_S = Beta('ASC_SCOOTER_S', 1, None, None, 0)
# ASC_SCOOTER_RND = ASC_SCOOTER + ASC_SCOOTER_S * bioDraws('ASC_SCOOTER_RND', 'NORMAL_ANTI')

# ASC_DLBIKE = Beta('ASC_DLBIKE', 0, None, None, 0)
# ASC_DLBIKE_S = Beta('ASC_DLBIKE_S', 1, None, None, 0)
# ASC_DLBIKE_RND = ASC_DLBIKE + ASC_DLBIKE_S * bioDraws('ASC_DLBIKE_RND', 'NORMAL_ANTI')

# ASC_DKBIKE = Beta('ASC_DKBIKE', 0, None, None, 0)
# ASC_DKBIKE_S = Beta('ASC_DKBIKE_S', 1, None, None, 0)
# ASC_DKBIKE_RND = ASC_DKBIKE + ASC_DKBIKE_S * bioDraws('ASC_DKBIKE_RND', 'NORMAL_ANTI')

# ASC_SCTRANSIT = Beta('ASC_SCTRANSIT', 0, None, None, 0)
# ASC_SCTRANSIT_S = Beta('ASC_SCTRANSIT_S', 1, None, None, 0)
# ASC_SCTRANSIT_RND = ASC_SCTRANSIT + ASC_SCTRANSIT_S * bioDraws('ASC_SCTRANSIT_RND', 'NORMAL_ANTI')

B_CARTIME = Beta('B_CARTIME', 0, None, None, 0)
B_TRANSITTIME = Beta('B_TRANSITTIME', 0, None, None, 0)
B_RHTIME = Beta('B_RHTIME', 0, None, None, 0)
B_WALKTIME = Beta('B_WALKTIME', 0, None, None, 0)
B_PERSBIKETIME = Beta('B_PERSBIKETIME', 0, None, None, 0)

B_SCOOTERTIME = Beta('B_SCOOTERTIME', 0, None, None, 0)
B_BIKETIME = Beta('B_BIKETIME', 0, None, None, 0)
# B_DKBIKETIME = Beta('B_DKBIKETIME', 0, None, None, 0)
B_SCTRANSITTIME = Beta('B_SCTRANSITTIME', 0, None, None, 0)

B_ACCESS = Beta('B_ACCESS', 0, None, None, 0)
B_DROP =Beta('B_DROP', 0, None, None, 0)
B_WAITAV = Beta('B_WAITAV', 0, None, None, 0)
B_AV = Beta('B_AV', 0, None, None, 0)

B_COST = Beta('B_COST', 0, None, None, 0)
B_COST_PRIME = Beta('B_COST_PRIME', 0, None, None, 0)
B_costadj = Beta('B_costadj', 0, None, None, 0)

B_PRCP_Yes = Beta('B_PRCP_Yes', 0, None, None, 0)
B_OWNBIKE = Beta('B_OWNBIKE', 0, None, None, 0)
B_BIKELANE = Beta('B_BIKELANE', 0, None, None, 0)
B_AGE_SQ = Beta('B_AGE_SQ', 0, None, None, 0)
B_AGE_SCTRANSIT_SQ = Beta('B_AGE_SCTRANSIT_SQ', 0, None, None, 0)
B_GENDER_F = Beta('B_GENDER_F', 0, None, None, 0)

B_AGE_MORE_35 = Beta('B_AGE_MORE_35', 0, None, None, 0)
B_CHILD_IN_HH = Beta('B_CHILD_IN_HH', 0, None, None, 0)
B_IS_WHITE = Beta('B_IS_WHITE', 0, None, None, 0)
B_FULLTIME_EMPLOY = Beta('B_FULLTIME_EMPLOY', 0, None, None, 0)
B_HHINCOME_MORE_75k = Beta('B_HHINCOME_MORE_75k', 0, None, None, 0)

In [7]:
# Nested logit terms, groupings = micromobility, conventional, active transportation, other
draw_micromobility = bioDraws('ASC_Micromobility', 'NORMAL_HALTON3')
draw_activetransp = bioDraws('ASC_ActiveTransp', 'NORMAL_HALTON3')
ASC_ACTIVETRANSP = Beta('ASC_ACTIVE TRANSP', 0, None, None, 0)
ASC_MICROMOBILITY = Beta('ASC_MICROMOBILITY', 0, None, None, 0)


ASC_WALK = Beta('ASC_WALK', 0, None, None, 0)
ASC_WALK_S = Beta('ASC_WALK_S', 1, None, None, 0)
ASC_WALK_RND = ASC_WALK + ASC_ACTIVETRANSP * draw_activetransp

ASC_BIKE = Beta('ASC_BIKE', 0, None, None, 0)
ASC_BIKE_S = Beta('ASC_BIKE_S', 1, None, None, 0)
ASC_BIKE_RND = ASC_BIKE + ASC_ACTIVETRANSP * draw_activetransp


# ASC_CAR = Beta('ASC_CAR', 0, None, None, 1)
# ASC_CAR_S = Beta('ASC_CAR_S', 0, None, None, 0)
# # ASC_CAR_RND = ASC_CAR #+ ASC_CAR_S * bioDraws('ASC_CAR_RND', 'NORMAL_HALTON3')

# ASC_TRANSIT = Beta('ASC_TRANSIT', 0, None, None, 0)
# ASC_TRANSIT_S = Beta('ASC_TRANSIT_S', 0, None, None, 0)
# ASC_TRANSIT_RND = ASC_TRANSIT # + ASC_TRANSIT_S * bioDraws('ASC_TRANSIT_RND', 'NORMAL_HALTON3')

# ASC_RH = Beta('ASC_RH', 0, None, None, 0)
# ASC_RH_S = Beta('ASC_RH_S', 0, None, None, 0)
# ASC_RH_RND = ASC_RH # ASC_RH_S * bioDraws('ASC_RH_RND', 'NORMAL_HALTON3')

# ASC_WALK = Beta('ASC_WALK', 0, None, None, 0)
# ASC_WALK_S = Beta('ASC_WALK_S', 0, None, None, 0)
# ASC_WALK_RND = ASC_WALK #+ ASC_ACTIVETRANSP * draw_activetransp #+ ASC_WALK_S * bioDraws('ASC_WALK_RND', 'NORMAL_HALTON3') +

# ASC_BIKE = Beta('ASC_BIKE', 0, None, None, 0)
# ASC_BIKE_RND = ASC_BIKE #+ ASC_ACTIVETRANSP * draw_activetransp

ASC_SCOOTER = Beta('ASC_SCOOTER', 0, None, None, 0)
ASC_SCOOTER_RND = ASC_SCOOTER + ASC_MICROMOBILITY * draw_micromobility + ASC_ACTIVETRANSP * draw_activetransp

ASC_DLBIKE = Beta('ASC_DLBIKE', 0, None, None, 0)
ASC_DLBIKE_RND = ASC_DLBIKE + ASC_MICROMOBILITY * draw_micromobility + ASC_ACTIVETRANSP * draw_activetransp 

ASC_DKBIKE = Beta('ASC_DKBIKE', 0, None, None, 0)
ASC_DKBIKE_RND = ASC_DKBIKE + ASC_MICROMOBILITY * draw_micromobility + ASC_ACTIVETRANSP * draw_activetransp

ASC_SCTRANSIT = Beta('ASC_SCTRANSIT', 0, None, None, 0)
ASC_SCTRANSIT_RND = ASC_SCTRANSIT + ASC_MICROMOBILITY * draw_micromobility + ASC_ACTIVETRANSP * draw_activetransp

In [8]:
B_MICRO_NONHOMEBASE = Beta('B_MICRO_NONHOMEBASE', 0, None, None, 0)
B_MICRO_HOMEBASE = Beta('B_MICRO_HOMEBASE', 0, None, None, 0)

B_TRANSIT_NONHOMEBASE = Beta('B_TRANSIT_NONHOMEBASE', 0, None, None, 0)
B_RH_NONHOMEBASE = Beta('B_RH_NONHOMEBASE', 0, None, None, 0)
B_WALK_NONHOMEBASE = Beta('B_WALK_NONHOMEBASE', 0, None, None, 0)
B_PERSBIKE_NONHOMEBASE = Beta('B_PERSBIKE_NONHOMEBASE', 0, None, None, 0)

B_TRANSIT_HOMEBASE = Beta('B_TRANSIT_HOMEBASE', 0, None, None, 0)
B_RH_HOMEBASE = Beta('B_RH_HOMEBASE', 0, None, None, 0)
B_WALK_HOMEBASE = Beta('B_WALK_HOMEBASE', 0, None, None, 0)
B_PERSBIKE_HOMEBASE = Beta('B_PERSBIKE_HOMEBASE', 0, None, None, 0)

In [9]:
# Utility functions
V1 = ASC_CAR + B_CARTIME * cartime

V2 = (
    ASC_TRANSIT
    + B_TRANSITTIME * transittime
    # + B_RISK_TRANSIT*PERCEIVEDRISK
    + B_TRANSIT_NONHOMEBASE * trippurp_NHB
    # + B_TRANSIT_HOMEBASE * trippurp_HBO
    # + B_TRANSIT_HOMEBASE * trippurp_HBSHOP
    # + B_TRANSIT_HOMEBASE * trippurp_HBSOC
)

V3 = (
    ASC_RH
    + B_RHTIME * rdtime
    # + B_RISK_RH*PERCEIVEDRISK
    + B_RH_NONHOMEBASE * trippurp_NHB
    # + B_RH_HOMEBASE * trippurp_HBO
    # + B_RH_HOMEBASE * trippurp_HBSHOP
    # + B_RH_HOMEBASE * trippurp_HBSOC
)

V4 = (
    ASC_WALK_RND
    + B_WALKTIME * walktime
    # + B_RISK_WALK*PERCEIVEDRISK
    #       + B_RISK_ACTIVETRANSP*PERCEIVEDRISK
    + B_WALK_NONHOMEBASE * trippurp_NHB
    # + B_WALK_HOMEBASE * trippurp_HBO
    # + B_WALK_HOMEBASE * trippurp_HBSHOP
    # + B_WALK_HOMEBASE * trippurp_HBSOC
)

V5 = (
    ASC_BIKE_RND
    + B_PERSBIKETIME * biketime
    # + B_RISK_PERSBIKE*PERCEIVEDRISK
    #       + B_RISK_ACTIVETRANSP*PERCEIVEDRISK
    + B_PERSBIKE_NONHOMEBASE * trippurp_NHB
    # + B_PERSBIKE_HOMEBASE * trippurp_HBO
    # + B_PERSBIKE_HOMEBASE * trippurp_HBSHOP
    # + B_PERSBIKE_HOMEBASE * trippurp_HBSOC
)

V7 = (
    ASC_SCOOTER_RND
    # + ASC_SCOOTER_PRIME*AVtech
    + B_SCOOTERTIME * sctime
    # + B_SCOOTERTIME_PRIME*sctime*AVtech
    + B_COST * sccost_adj
    # + B_COST_PRIME*sccost_adj*AVtech
    + B_ACCESS * SCAW * (1 - AVtech)
    + B_WAITAV * SCAV * AVtech
    + B_PRCP_Yes * PRCP_YES
    + B_BIKELANE * BKLN_YES
    + B_MICRO_NONHOMEBASE * trippurp_NHB
    # + B_MICRO_HOMEBASE * trippurp_HBO
    # + B_MICRO_HOMEBASE * trippurp_HBSHOP
    # + B_MICRO_HOMEBASE * trippurp_HBSOC
    # # + B_RISK_SCOOTER*PERCEIVEDRISK
    #       + B_RISK_MICROMOBILITY*PERCEIVEDRISK
    # + B_NORM_MICROMOBILITY_SCOOTER*SOCIALNORM
    # # + B_ENVVALUE_MICROMOBILITY_SCOOTER * ENVVALUE
    # + B_ENVVALUE_MICROMOBILITY * ENVVALUE
)
V8 = (
    ASC_DLBIKE_RND
    + B_BIKETIME * dltime
    + B_COST * dlcost_adj
    + B_ACCESS * DLAW
    + B_PRCP_Yes * PRCP_YES
    + B_BIKELANE * BKLN_YES
    + B_MICRO_NONHOMEBASE * trippurp_NHB
    # + B_MICRO_HOMEBASE * trippurp_HBO
    # + B_MICRO_HOMEBASE * trippurp_HBSHOP
    # + B_MICRO_HOMEBASE * trippurp_HBSOC
    #         + B_RISK_MICROMOBILITY*PERCEIVEDRISK
    #   + B_NORM_MICROMOBILITY_BIKE*SOCIALNORM
    # + B_RISK_BIKE*PERCEIVEDRISK
    # + B_ENVVALUE_MICROMOBILITY_BIKE * ENVVALUE
    #    + B_ENVVALUE_MICROMOBILITY * ENVVALUE
)
V9 = (
    ASC_DKBIKE_RND
    + B_BIKETIME * dbtime
    + B_COST * dbcost_adj
    + B_ACCESS * DBAW
    + B_DROP * DBDW
    + B_PRCP_Yes * PRCP_YES
    + B_BIKELANE * BKLN_YES
    + B_MICRO_NONHOMEBASE * trippurp_NHB
    # + B_MICRO_HOMEBASE * trippurp_HBO
    # + B_MICRO_HOMEBASE * trippurp_HBSHOP
    # + B_MICRO_HOMEBASE * trippurp_HBSOC
    # + B_RISK_BIKE*PERCEIVEDRISK
    # + B_NORM_MICROMOBILITY_BIKE*SOCIALNORM
    # # + B_ENVVALUE_MICROMOBILITY_BIKE * ENVVALUE
    #    + B_ENVVALUE_MICROMOBILITY * ENVVALUE
)
V10 = (
    ASC_SCTRANSIT_RND
    + B_SCTRANSITTIME * sttotime
    + B_COST * sttocost_adj
    + B_ACCESS * STAW * (1 - AVtech)
    + B_WAITAV * STAV * AVtech
    + B_PRCP_Yes * PRCP_YES
    + B_BIKELANE * BKLN_YES
    + B_MICRO_NONHOMEBASE * trippurp_NHB
    # + B_MICRO_HOMEBASE * trippurp_HBO
    # + B_MICRO_HOMEBASE * trippurp_HBSHOP
    # + B_MICRO_HOMEBASE * trippurp_HBSOC
    #  + B_RISK_SCTRANSIT*PERCEIVEDRISK
    #  + B_NORM_MICROMOBILITY_SCOOTER*SOCIALNORM
    # # + B_ENVVALUE_MICROMOBILITY_SCOOTER * ENVVALUE
    #    + B_ENVVALUE_MICROMOBILITY * ENVVALUE
)

In [10]:
V = {1: V1, 2: V2, 3: V3, 4: V4, 5: V5, 7: V7, 8: V8, 9: V9, 10: V10}
av = {1: car_av, 2: transit_av, 3: rd_av, 4: walk_av, 5: bike_av, 
      7: scooter_av, 8: dlbike_av, 9: dkbike_av, 10: sctransit_av}
      
# condprob = models.logit(V, av, choice)
# condprobindiv = PanelLikelihoodTrajectory(condprob)
# condlike = (
#     P_Risk01 * 
#     P_Risk02 * 
#     P_Risk03 * 
#     #P_Risk04 *
#     condprobindiv
# )
# logprob = log(MonteCarlo(condprobindiv))

#define nests - 1: nests parameter, 2: list of alternatives


condprob = models.logit(V, av, choice)
# condlike = (
#     P_Norm01 *
#     P_Norm02 *
#     P_Norm03 +
#     condprob
# )
condlikeindiv = PanelLikelihoodTrajectory(condprob)
logprob = log(MonteCarlo(condlikeindiv))

biogeme = bio.BIOGEME(database, logprob, numberOfDraws = 500, numberOfThreads = 100)

# biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = "Nested Logit with trip purpose, precipitation - 250 Draws"
# biogeme.modelName = "HybridChoiceNestedLogit"

results = biogeme.estimate()
pandasResults = results.getEstimatedParameters()
print(f"Estimated betas: {len(results.data.betaValues)}")
print(f"Final log likelihood: {results.data.logLike:.3f}")
print(pandasResults.round(2))

The use of argument numberOfThreads in the constructor of the BIOGEME object is deprecated and will be removed in future versions of Biogeme. Instead, define parameter number_of_threads in section MultiThreadingof the .toml parameter file. The default file name is biogeme.toml
The use of argument numberOfDraws in the constructor of the BIOGEME object is deprecated and will be removed in future versions of Biogeme. Instead, define parameter number_of_draws in section MonteCarloof the .toml parameter file. The default file name is biogeme.toml


Estimated betas: 29
Final log likelihood: -5361.643
                        Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_ACTIVE TRANSP       -0.03          0.29        -0.10          0.92
ASC_BIKE                -2.96          0.42        -7.00          0.00
ASC_DKBIKE              -3.01          0.33        -9.05          0.00
ASC_DLBIKE              -2.43          0.32        -7.61          0.00
ASC_MICROMOBILITY        2.83          0.25        11.17          0.00
ASC_RH                  -1.20          0.76        -1.59          0.11
ASC_SCOOTER             -2.48          0.30        -8.30          0.00
ASC_SCTRANSIT           -3.23          0.23       -13.96          0.00
ASC_TRANSIT             -1.68          0.86        -1.95          0.05
ASC_WALK                -2.63          0.62        -4.26          0.00
B_ACCESS                -0.10          0.01        -7.16          0.00
B_BIKELANE              -0.18          0.12        -1.50          0.13
B_BIKETIME              -

In [ ]:
# corr = calculate_correlation(
#     nests, results, alternative_names = {1: 'Car', 2: 'Transit', 3: 'Ridehailing', 4: 'Walk', 5: 'Personal Bike',
#                                          7: 'Scooter', 8: 'Dockless Bike', 9: 'Docked Bike', 10: 'Scooter + Transit'}
# )
# print(corr)